# Neural Network

## Imports

In [ ]:
import itertools
import os
from typing import Tuple, List

import numpy as np
import pandas as pd
import torch
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from torch import nn
from torch import utils
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torchinfo import summary

from src.data.dataset import MovieDataset
from src.models.config import best_param_layers, best_param_grid_mlp
from src.models.network.mlp import execute
from src.models.network.validate import validate, test_eval
from src.utils.const import DATA_DIR, SEED, NUM_BINS
from src.utils.util_models import fix_random, balancer

### Useful path to data

In [ ]:
ROOT_DIR = os.path.join(os.getcwd(), '..')
PROCESSED_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'processed')

### Fix random seed

In [ ]:
fix_random(SEED)

### Set device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print('Using device:', torch.cuda.get_device_name(device))

### Import final dataset

In [ ]:
final_stored = pd.read_parquet(os.path.join(PROCESSED_DIR, 'final.parquet'))
final = MovieDataset(final_stored)

## Dataset

In [ ]:
class MovieDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        self.idx_column = {}
        for idx, col_name in enumerate(df.columns):
            self.idx_column[col_name] = idx

        X, y_continuous = self.data_target_split(df)

        self.num_classes = NUM_BINS
        y = self._discretize(y_continuous)

        self.X = torch.tensor(X, dtype=torch.float)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self) -> int:
        return self.X.shape[0]

    def __getitem__(self, idx: int) -> Tuple:
        return self.X[idx, :], self.y[idx]

    @staticmethod
    def data_target_split(df: pd.DataFrame) -> Tuple:
        y = df['rating_mean']
        X = df.drop(columns='rating_mean').to_numpy()
        return X, y

    def _discretize(self, target: pd.Series) -> pd.Series:
        y = pd.cut(target, bins=self.num_classes, labels=False)
        return y

    def scale(self, train_idx, test_idx, scaler, features: List[int]):
        train_data = self.X[train_idx]
        test_data = self.X[test_idx]

        for feature in features:
            feature_train = train_data[:, feature].reshape(-1, 1)
            feature_test = test_data[:, feature].reshape(-1, 1)

            scaled_train = np.squeeze(scaler.fit_transform(feature_train))
            scaled_test = np.squeeze(scaler.transform(feature_test))

            self.X[train_idx, feature] = torch.tensor(scaled_train, dtype=torch.float)
            self.X[test_idx, feature] = torch.tensor(scaled_test, dtype=torch.float)

    def normalize(self, train_idx, test_idx, norm: str = 'l2'):
        train_data = self.X[train_idx]
        test_data = self.X[test_idx]

        norm_train = normalize(train_data, norm=norm)
        norm_test = normalize(test_data, norm=norm)

        self.X[train_idx, :] = torch.tensor(norm_train, dtype=torch.float)
        self.X[test_idx, :] = torch.tensor(norm_test, dtype=torch.float)

## Architecture

In [ ]:
class MovieNet(nn.Module):
    def __init__(
            self,
            input_size: int,
            input_act: nn.Module,
            hidden_size: int,
            hidden_act: nn.Module,
            num_hidden_layers: int,
            output_fn,
            num_classes: int,
            dropout: float = 0.0,
            batch_norm: bool = False
    ) -> None:
        super(MovieNet, self).__init__()

        self.layers = nn.ModuleList([
            nn.Linear(input_size, hidden_size),
            input_act
        ])

        for _ in range(num_hidden_layers):
            self.layers.append(nn.Linear(hidden_size, hidden_size))

            if batch_norm:
                self.layers.append(nn.BatchNorm1d(hidden_size))

            self.layers.append(hidden_act)

            if dropout > 0.0:
                self.layers.append(nn.Dropout(dropout))

        self.layers.append(nn.Linear(hidden_size, num_classes))

        if output_fn:
            self.layers.append(output_fn)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def reset_weights(self):
        for layer in self.layers:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

## Train & Test

In [ ]:
def train_test(dataset: MovieDataset):
    features = [
        dataset.idx_column['year'],
        dataset.idx_column['title_length'],
        dataset.idx_column['tag_count'],
        dataset.idx_column['runtime'],
        dataset.idx_column['rating_count']
    ]
    num_workers = 2

    n_splits = 5
    cv_outer = StratifiedKFold(n_splits=n_splits, shuffle=True)

    for fold, (train_idx, test_idx) in enumerate(cv_outer.split(dataset.X, y=dataset.y), 1):
        hyper_parameters_model_all = itertools.product(
            best_param_layers['input_act'],
            best_param_layers['hidden_act'],
            best_param_layers['hidden_size'],
            best_param_layers['num_hidden_layers'],
            best_param_layers['dropout'],
            best_param_layers['batch_norm'],
            best_param_layers['output_fn'],
            best_param_grid_mlp['starting_lr'],
            best_param_grid_mlp['num_epochs'],
            best_param_grid_mlp['batch_size'],
            best_param_grid_mlp['optim'],
            best_param_grid_mlp['momentum'],
            best_param_grid_mlp['weight_decay']
        )

        hyper_parameters_model = hyper_parameters_model_all

        print('=' * 65)
        print(f'Fold {fold}')

        data_test = utils.data.Subset(dataset, test_idx)

        loader_test = utils.data.DataLoader(data_test, batch_size=1,
                                            shuffle=False,
                                            num_workers=num_workers)

        for idx, (input_act,
                  hidden_act,
                  hidden_size,
                  num_hidden_layers,
                  dropout,
                  batch_norm,
                  _,
                  starting_lr,
                  num_epochs,
                  batch_size,
                  optimizer_class,
                  momentum,
                  weight_decay) in enumerate(hyper_parameters_model):

            cfg = (
                input_act, hidden_act, hidden_size, num_hidden_layers, dropout, batch_norm, starting_lr, num_epochs,
                batch_size, optimizer_class, momentum, weight_decay)

            cv_inner = StratifiedKFold(n_splits=n_splits, shuffle=True)

            for inner_fold, (inner_train_idx, val_idx) in enumerate(
                    cv_inner.split(dataset.X[train_idx], y=dataset.y[train_idx]), 1):

                # Balancing
                train_target = dataset.y[inner_train_idx]
                sampler = balancer(train_target)

                # Scaling
                scaler = preprocessing.MinMaxScaler()
                dataset.scale(train_idx, test_idx, scaler, features)

                data_train = utils.data.Subset(dataset, inner_train_idx)
                data_val = utils.data.Subset(dataset, val_idx)

                loader_train = utils.data.DataLoader(data_train, batch_size=batch_size,
                                                     sampler=sampler,
                                                     pin_memory=True,
                                                     num_workers=num_workers)

                loader_val = utils.data.DataLoader(data_val, batch_size=1,
                                                   shuffle=False,
                                                   num_workers=num_workers)

                input_size = dataset.X.shape[1]
                num_classes = dataset.num_classes
                network = MovieNet(input_size=input_size,
                                   input_act=input_act,
                                   hidden_size=hidden_size,
                                   hidden_act=hidden_act,
                                   num_hidden_layers=num_hidden_layers,
                                   dropout=dropout,
                                   output_fn=None,
                                   num_classes=num_classes)
                network.reset_weights()
                network.to(device)

                if fold == 1 and inner_fold == 1:
                    print('=' * 65)
                    print(f'Configuration [{idx}]: {cfg}')
                    summary(network)

                # TODO: fix experiment name
                name_train = f'movie_net_experiment_{idx}'

                if optimizer_class == torch.optim.Adam:
                    optimizer = optimizer_class(network.parameters(),
                                                lr=starting_lr,
                                                weight_decay=weight_decay)
                else:
                    optimizer = optimizer_class(network.parameters(),
                                                lr=starting_lr,
                                                momentum=momentum,
                                                weight_decay=weight_decay)

                fold_stat = execute(name_train,
                                    network,
                                    optimizer,
                                    num_epochs,
                                    loader_train,
                                    loader_val,
                                    device)

            criterion = CrossEntropyLoss()
            loss_test, acc_test, f1_test = test_eval(fold, loader_test, device, criterion, True)
            print(f'Test {fold}, loss={loss_test:3f}, accuracy={acc_test:3f}, f1={f1_test:3f}')

In [ ]:
train_test(final)